In [3]:
# Load data
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_d110e2fe011d44d489dffcc5194a59c5 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='eb_teHef1zl2WqhjGL8xoD85lSXUh6_9VurXa8loCIlM',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_d110e2fe011d44d489dffcc5194a59c5.get_object(Bucket='sepsisprediction-donotdelete-pr-wodmbegr9uodno',Key='data_without_missing_value.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()



,住院流水号,时间戳,C反应蛋白,白细胞数量,国际标准化比值,血浆降钙素原,血浆总胆红素,血小板,血氨,碱性磷酸酶,...,钙,肌酐,纤维蛋白原,血红蛋白,血乳酸,镁,磷,钾,二氧化碳分压,血沉
0,2017-02176829-0,3,35.109718,14.201427,1.304449,1.471463,6.70000,356.877049,60.600000,405.000000,...,1.780000,16.00000,3.174734,206.406158,5.100000,0.906821,1.471463,5.260000,33.752776,NaN
1,2017-02176829-0,4,35.109718,11.700000,0.850000,1.471463,6.70000,567.000000,60.600000,405.000000,...,1.780000,16.00000,3.040000,219.000000,5.100000,0.906821,1.471463,5.260000,33.752776,NaN
2,2017-02175870-0,4,35.109718,14.201427,1.304449,1.471463,25.86681,356.877049,52.959677,142.766513,...,1.836576,32.93266,3.174734,206.406158,2.646719,0.906821,1.471463,4.087293,38.526478,NaN
3,2017-02175870-0,21,35.109718,14.201427,1.304449,1.471463,25.86681,356.877049,52.959677,142.766513,...,1.836576,32.93266,3.174734,206.406158,2.646719,0.906821,1.471463,4.087293,38.526478,NaN
4,2017-02175870-0,40,35.109718,14.201427,1.304449,1.471463,25.86681,356.877049,52.959677,142.766513,...,1.836576,32.93266,3.174734,206.406158,2.646719,0.906821,1.471463,4.087293,38.526478,NaN


In [17]:
# Expand data, i.e fill every 1 hour bin
data = data.sort_values([u'住院流水号', u'时间戳'])

pre_patient = None
pre_time = None

new_data = pd.DataFrame()
n_patient = 0
for i in data.index:
    line = data.iloc[i]
    patient_no, time_stamp = line[u'住院流水号'], line[u'时间戳']
    if pre_patient is not None:
        if pre_patient == patient_no:
            t_diff = time_stamp - pre_time
            for t in range(1, t_diff):
                new_data = new_data.append(pd.DataFrame({u'住院流水号': [patient_no],u'时间戳': [pre_time+t]}))
        else:
            n_patient += 1
            if n_patient % 50 == 0:
                print("Processed %d patients" % n_patient)
    else:
        n_patient += 1
        if n_patient % 50 == 0:
            print("Processed %d patients" % n_patient)
    pre_patient = patient_no
    pre_time = time_stamp
    new_data = new_data.append(line)
    
new_data.head()

Processed 50 patients
Processed 100 patients
Processed 150 patients
Processed 200 patients
Processed 250 patients
Processed 300 patients
Processed 350 patients


,C反应蛋白,二氧化碳分压,住院流水号,国际标准化比值,天门冬氨酸氨基转氨酶,时间戳,白细胞数量,直接胆红素,碱性磷酸酶,磷,...,血乳酸,血小板,血氨,血沉,血浆总胆红素,血浆降钙素原,血红蛋白,钙,钾,镁
0,35.109718,33.752776,2017-02176829-0,1.304449,74.000000,3.0,14.201427,4.700000,405.000000,1.471463,...,5.100000,356.877049,60.600000,NaN,6.70000,1.471463,206.406158,1.780000,5.260000,0.906821
1,35.109718,33.752776,2017-02176829-0,0.850000,74.000000,4.0,11.700000,4.700000,405.000000,1.471463,...,5.100000,567.000000,60.600000,NaN,6.70000,1.471463,219.000000,1.780000,5.260000,0.906821
2,35.109718,38.526478,2017-02175870-0,1.304449,211.669578,4.0,14.201427,14.911079,142.766513,1.471463,...,2.646719,356.877049,52.959677,NaN,25.86681,1.471463,206.406158,1.836576,4.087293,0.906821
0,NaN,NaN,2017-02175870-0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,2017-02175870-0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
new_data = new_data.sort_values([u'住院流水号', u'时间戳'])
new_data = new_data.fillna(method='ffill')
new_data.head()

,C反应蛋白,二氧化碳分压,住院流水号,国际标准化比值,天门冬氨酸氨基转氨酶,时间戳,白细胞数量,直接胆红素,碱性磷酸酶,磷,...,血乳酸,血小板,血氨,血沉,血浆总胆红素,血浆降钙素原,血红蛋白,钙,钾,镁
1828,35.109718,38.526478,2016-0682617-0,1.304449,211.669578,3.0,2.7,14.911079,142.766513,1.471463,...,2.646719,542.0,33.0,NaN,25.86681,1.471463,204.5,1.836576,4.087293,0.906821
1829,35.109718,48.378978,2016-0682617-0,1.450000,35.000000,4.0,2.7,1.700000,48.000000,1.471463,...,1.100000,542.0,33.0,NaN,3.80000,1.471463,69.5,1.184000,3.350000,0.750000
0,35.109718,48.378978,2016-0682617-0,1.450000,35.000000,5.0,2.7,1.700000,48.000000,1.471463,...,1.100000,542.0,33.0,NaN,3.80000,1.471463,69.5,1.184000,3.350000,0.750000
0,35.109718,48.378978,2016-0682617-0,1.450000,35.000000,6.0,2.7,1.700000,48.000000,1.471463,...,1.100000,542.0,33.0,NaN,3.80000,1.471463,69.5,1.184000,3.350000,0.750000
0,35.109718,48.378978,2016-0682617-0,1.450000,35.000000,7.0,2.7,1.700000,48.000000,1.471463,...,1.100000,542.0,33.0,NaN,3.80000,1.471463,69.5,1.184000,3.350000,0.750000


In [20]:
# Save data to the storage
from project_lib import Project
project = Project(project_id="54710414-eeb2-42df-820a-698b93f50f84", project_access_token="p-594d39b7f91d72f663da93ed52c36b329ac8c2c7")
project.save_data('data_1_hour_bin.csv', new_data.to_csv(header=True, index=False), overwrite=True)

{'asset_id': 'ccce97c4-65ba-4b78-9cc9-a77f1a0e86f2',
 'bucket_name': 'sepsisprediction-donotdelete-pr-wodmbegr9uodno',
 'file_name': 'data_1_hour_bin.csv',
 'message': 'File data_1_hour_bin.csv has been written successfully to the associated OS'}

In [4]:

body = client_d110e2fe011d44d489dffcc5194a59c5.get_object(Bucket='sepsisprediction-donotdelete-pr-wodmbegr9uodno',Key='target.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

target = pd.read_csv(body)
target.head()



,住院流水号,时间戳,诊断结果
0,2016-0682617-0,26,0
1,2016-0682617-0,68,0
2,2016-0682718-0,68,0
3,2016-0683383-0,2,0
4,2016-0683383-0,18,0


In [7]:
target = target.sort_values([u'住院流水号', u'时间戳'])
target.head(20)

,住院流水号,时间戳,诊断结果
290,2016-0682617-0,3,0
434,2016-0682617-0,4,1
144,2016-0682617-0,21,0
145,2016-0682617-0,24,0
0,2016-0682617-0,26,0
291,2016-0682617-0,51,0
146,2016-0682617-0,53,0
147,2016-0682617-0,60,0
1,2016-0682617-0,68,0
148,2016-0682617-0,69,0


In [12]:
# Expand target, i.e fill every 1 hour bin
target = target.sort_values([u'住院流水号', u'时间戳'])

pre_patient = None
pre_time = None

new_target = pd.DataFrame()
n_patient = 0
patient_ids = target[u'住院流水号'].unique()
for patient_id in patient_ids:
    patient_data = target[target[u'住院流水号'] == patient_id].sort_values(u'时间戳').reindex()
    for i in patient_data.index:
        print(i)
            
'''
for i in target.sort_values([u'住院流水号', u'时间戳']).index:
    line = target.iloc[i]
    patient_no, time_stamp = line[u'住院流水号'], line[u'时间戳']
    if pre_patient is not None:
        if pre_patient == patient_no:
            t_diff = time_stamp - pre_time
            for t in range(1, t_diff):
                new_target = new_target.append(pd.DataFrame({u'住院流水号': [patient_no],u'时间戳': [pre_time+t]}))
        else:
            n_patient += 1
            print(patient_no, pre_patient)
            if n_patient % 50 == 0:
                print("Processed %d patients" % n_patient)
    else:
        n_patient += 1
        print(patient_no, pre_patient)
        if n_patient % 50 == 0:
            print("Processed %d patients" % n_patient)
    pre_patient = patient_no
    pre_time = time_stamp
    new_target = new_target.append(line)
    
new_target = new_target.sort_values([u'住院流水号', u'时间戳'])
new_target = new_target.fillna(method='ffill')
new_target.head()
'''

290
434
144
145
0
291
146
147
1
148
149
292
150
293
435
151
152
153
294
436
295
437
2
154
296
155
156
438
297
3
298
4
157
5
158
439
440
299
441
300
442
443
6
7
159
160
444
301
302
303
161
304
445
8
9
446
305
306
447
307
162
163
164
308
309
165
310
10
448
311
312
313
314
449
315
11
450
316
166
317
12
318
451
452
319
453
167
168
13
169
320
14
454
455
15
321
170
322
171
456
323
324
325
457
326
172
173
327
16
17
18
19
174
328
175
329
176
177
458
20
178
21
179
330
459
180
331
460
332
181
182
22
333
23
183
24
184
25
461
334
185
335
462
186
187
188
189
336
190
337
26
338
463
339
27
28
191
340
464
29
341
30
31
342
192
343
465
466
344
467
32
468
193
33
194
345
195
469
34
470
346
347
471
196
35
348
36
472
349
197
473
474
37
475
350
351
198
199
352
476
200
477
38
39
40
201
353
478
479
202
354
41
480
203
204
481
205
42
206
355
43
482
356
483
207
44
208
209
484
45
210
46
357
358
211
359
485
360
486
212
361
487
213
47
488
214
215
216
48
49
489
490
217
362
50
218
363
219
220
221
51
364
222
365
366
52

2598
1359
2599
2393
1360
1791
2394
2600
2395
2396
2601
2602
2603
2397
2604
2398
2605
2399
2606
1361
2400
1792
1793
2401
1362
2607
2402
2608
2609
1363
1794
2610
1795
2403
2611
2404
1796
1364
1365
1797
2405
1798
2406
1366
2612
2407
1799
1800
2613
1801
1802
2408
2409
2614
1803
1804
1367
2410
1368
1805
2615
1806
1369
1370
2616
1807
1808
1371
2617
1809
1810
1811
1372
2411
2412
2618
1812
2413
2414
1813
1814
1373
2619
2620
2621
1374
2622
2623
1375
2415
1376
1815
2416
2417
1816
1377
2418
1817
1378
2624
1818
2625
1379
1380
2419
2626
1819
1820
2420
1821
1381
2421
2627
1822
1823
1382
2422
1824
1825
2628
1383
2423
2424
2629
2630
1384
2631
2425
1826
2632
2633
2426
2427
1827
2428
2429
1828
2634
2430
2635
2431
1385
2636
1386
2432
2433
1829
2434
1830
1831
1387
1388
1832
1833
2435
2436
2437
2438
1834
1389
1835
1836
1390
1391
1837
2439
1392
1838
1393
2440
1394
2637
1395
2638
2441
1839
2442
2443
1396
1840
1841
2444
1397
1842
2445
2639
2640
2641
1398
1399
1400
2446
2642
1843
1401
1844
1402
2643
1845
1403


'\nfor i in target.sort_values([u\'住院流水号\', u\'时间戳\']).index:\n    line = target.iloc[i]\n    patient_no, time_stamp = line[u\'住院流水号\'], line[u\'时间戳\']\n    if pre_patient is not None:\n        if pre_patient == patient_no:\n            t_diff = time_stamp - pre_time\n            for t in range(1, t_diff):\n                new_target = new_target.append(pd.DataFrame({u\'住院流水号\': [patient_no],u\'时间戳\': [pre_time+t]}))\n        else:\n            n_patient += 1\n            print(patient_no, pre_patient)\n            if n_patient % 50 == 0:\n                print("Processed %d patients" % n_patient)\n    else:\n        n_patient += 1\n        print(patient_no, pre_patient)\n        if n_patient % 50 == 0:\n            print("Processed %d patients" % n_patient)\n    pre_patient = patient_no\n    pre_time = time_stamp\n    new_target = new_target.append(line)\n    \nnew_target = new_target.sort_values([u\'住院流水号\', u\'时间戳\'])\nnew_target = new_target.fillna(method=\'ffill\')\nnew_target.hea

In [10]:
# Obtain training data, i.e. a sequence of tuples in the format of (x, t, s),
# where x is a vector of features, t is the time to the occurrence of sepsis 
# and s represents whether the event happended, if happend it is 1 otherwise 0.
window_size = 4
n_row = len(new_data)

train_data = []
for i in range(n_row):
    line = new_data.iloc[i]
    patient_no, time_stamp = line[u'住院流水号'], line[u'时间戳']
    x = line.drop([u'住院流水号', u'时间戳'], axis=1)
    print(x)
    s = -1
    for t in range(time_stamp, time_stamp + window_size + 1):
        if target[(patient_no, t), u'诊断结果'] == 1:
            tao = t - time_stamp
            s = 1
            break
    if s == -1:
        tao = window_size + 1
        s = 0
    train_data.append(x, tao, s)
    
        
    
    

NameError: name 'new_data' is not defined